# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [5]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

/Users/miwa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 演習問題

---
> P-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

In [5]:
def label_age(x):
    if x >= 60 :
        return '60歳代'
    else:
        return str(x//10) + '0歳代'
    
df_customer_samary = pd.concat([df_customer[['customer_id', 'birth_day']], df_customer['age'].apply(label_age)], axis=1)
df_customer_samary.head(10)

,customer_id,birth_day,age
0,CS021313000114,1981-04-29,30歳代
1,CS037613000071,1952-04-01,60歳代
2,CS031415000172,1976-10-04,40歳代
3,CS028811000001,1933-03-27,60歳代
4,CS001215000145,1995-03-29,20歳代
5,CS020401000016,1974-09-15,40歳代
6,CS015414000103,1977-08-09,40歳代
7,CS029403000008,1973-08-17,40歳代
8,CS015804000004,1931-05-02,60歳代
9,CS033513000180,1962-07-11,50歳代


---
> P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

In [9]:
df_customer_samary['gender'] = df_customer['gender']
pd.pivot_table(df_customer_samary, index='gender', columns='age', values='customer_id', aggfunc='count')

age,10歳代,20歳代,30歳代,40歳代,50歳代,60歳代
gender,,,,,,
不明,9,52,188,288,280,255
女性,144,1032,3134,4876,4347,4385
男性,15,110,424,759,798,875


---
> P-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

In [14]:
def label_damy(x):
    if x == 1 :
        return 1
    else:
        return 0

df_customer_damy = pd.concat([df_customer[['customer_id']], df_customer['gender_cd'].apply(label_damy)], axis=1)
df_customer_damy.head(10)

,customer_id,gender_cd
0,CS021313000114,1
1,CS037613000071,0
2,CS031415000172,1
3,CS028811000001,1
4,CS001215000145,1
5,CS020401000016,0
6,CS015414000103,1
7,CS029403000008,0
8,CS015804000004,0
9,CS033513000180,1


---
> P-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [19]:
# 答えみた

df_receipt_samary = df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount.sum().reset_index()
df_receipt_samary['amount_ss'] = preprocessing.scale(df_receipt_samary['amount'])
df_receipt_samary.head(10)

/Users/miwa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


,customer_id,amount,amount_ss
0,CS001113000004,1298,-0.459378
1,CS001114000005,626,-0.706390
2,CS001115000010,3044,0.182413
3,CS001205000004,1988,-0.205749
4,CS001205000006,3337,0.290114
5,CS001211000025,456,-0.768879
6,CS001212000027,448,-0.771819
7,CS001212000031,296,-0.827691
8,CS001212000046,228,-0.852686
9,CS001212000070,456,-0.768879


---
> P-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [22]:
df_receipt_samary = df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount.sum().reset_index()
df_receipt_samary['amount_mm'] = preprocessing.minmax_scale(df_receipt_samary['amount'])
df_receipt_samary.head(10)

/Users/miwa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


,customer_id,amount,amount_mm
0,CS001113000004,1298,0.053354
1,CS001114000005,626,0.024157
2,CS001115000010,3044,0.129214
3,CS001205000004,1988,0.083333
4,CS001205000006,3337,0.141945
5,CS001211000025,456,0.016771
6,CS001212000027,448,0.016423
7,CS001212000031,296,0.009819
8,CS001212000046,228,0.006865
9,CS001212000070,456,0.016771
